# Capítulo 05 - Importação e Exportação de JSON

Este notebook explora o trabalho com arquivos JSON no DuckDB, incluindo JSON simples, NDJSON (newline-delimited JSON), e estruturas aninhadas.

## 📚 Tópicos Abordados:
1. Introdução ao JSON
2. Instalação e Preparação
3. Leitura de JSON Simples
4. Leitura de NDJSON (Newline Delimited)
5. Exportação para JSON
6. Trabalhando com Estruturas Aninhadas
7. JSON de Dicionários Python
8. Arrays e Listas em JSON
9. Consultas Complexas em JSON
10. Conversão JSON ↔ CSV ↔ Parquet
11. Performance e Boas Práticas

## 1. Introdução ao JSON

### O que é JSON?
- **JavaScript Object Notation**: Formato de dados em texto
- **Estruturado**: Chaves e valores
- **Legível**: Fácil para humanos e máquinas
- **Universal**: Usado em APIs, configurações, etc.

### Tipos de JSON:
1. **JSON Array**: `[{...}, {...}]` - Lista de objetos
2. **NDJSON**: Cada linha é um objeto JSON separado
3. **JSON Aninhado**: Objetos dentro de objetos

## 2. Instalação e Preparação

In [ ]:
# Instalar dependências
%pip install duckdb pandas -q
print("✓ Pacotes instalados!")

In [ ]:
import duckdb
import json
import os

print(f"DuckDB versão: {duckdb.__version__}")
print("\n✓ Imports realizados!")

## 3. Leitura de JSON Simples

### 3.1 Criar Arquivo JSON de Teste

In [ ]:
# Criar JSON array
dados_json = [
    {"id": 1, "nome": "Alice", "idade": 28, "cidade": "São Paulo", "salario": 5500.00},
    {"id": 2, "nome": "Bob", "idade": 35, "cidade": "Rio de Janeiro", "salario": 7200.00},
    {"id": 3, "nome": "Charlie", "idade": 42, "cidade": "Belo Horizonte", "salario": 6800.00},
    {"id": 4, "nome": "Diana", "idade": 31, "cidade": "Curitiba", "salario": 5900.00},
    {"id": 5, "nome": "Eduardo", "idade": 29, "cidade": "Porto Alegre", "salario": 6100.00}
]

with open('example.json', 'w', encoding='utf-8') as f:
    json.dump(dados_json, f, ensure_ascii=False, indent=2)

print("✓ Arquivo 'example.json' criado!")
print(f"Tamanho: {os.path.getsize('example.json'):,} bytes")

# Mostrar conteúdo
print("\nPrimeiras linhas:")
with open('example.json', 'r', encoding='utf-8') as f:
    print(f.read()[:200] + "...")

### 3.2 Ler com `read_json()`

In [ ]:
# Ler JSON como Relation
rel = duckdb.read_json("example.json")

print("Tipo do objeto:", type(rel))
print("\nDados:")
rel.show()

print("\n✓ JSON lido com sucesso!")

### 3.3 Query SQL Direta

In [ ]:
# Query direta no arquivo JSON
print("Consulta SQL direta:")
result = duckdb.sql("SELECT * FROM 'example.json'")
result.show()

print("\n✓ Query executada!")

### 3.4 Filtros e Agregações

In [ ]:
# Filtrar dados
print("Pessoas com idade > 30:")
duckdb.sql("""
    SELECT nome, idade, cidade, salario
    FROM 'example.json'
    WHERE idade > 30
    ORDER BY salario DESC
""").show()

# Estatísticas
print("\nEstatísticas:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total,
        AVG(idade) AS idade_media,
        AVG(salario) AS salario_medio,
        MAX(salario) AS max_salario
    FROM 'example.json'
""").show()

## 4. Leitura de NDJSON (Newline Delimited)

NDJSON é um formato onde cada linha é um objeto JSON separado.

### 4.1 Criar Arquivo NDJSON

In [ ]:
# Criar NDJSON (cada linha = 1 objeto)
ndjson_lines = [
    '{"produto": "Notebook", "categoria": "Eletrônicos", "preco": 3500.00, "estoque": 15}',
    '{"produto": "Mouse", "categoria": "Eletrônicos", "preco": 50.00, "estoque": 120}',
    '{"produto": "Teclado", "categoria": "Eletrônicos", "preco": 150.00, "estoque": 80}',
    '{"produto": "Cadeira", "categoria": "Móveis", "preco": 899.00, "estoque": 25}',
    '{"produto": "Mesa", "categoria": "Móveis", "preco": 1200.00, "estoque": 10}'
]

with open('produtos.ndjson', 'w', encoding='utf-8') as f:
    f.write('\n'.join(ndjson_lines))

print("✓ Arquivo 'produtos.ndjson' criado!")
print(f"Tamanho: {os.path.getsize('produtos.ndjson'):,} bytes")

# Mostrar conteúdo
print("\nConteúdo:")
with open('produtos.ndjson', 'r') as f:
    for i, linha in enumerate(f, 1):
        print(f"Linha {i}: {linha.strip()[:60]}...")

### 4.2 Ler NDJSON

In [ ]:
# DuckDB detecta automaticamente NDJSON
print("Lendo NDJSON:")
duckdb.sql("SELECT * FROM 'produtos.ndjson'").show()

print("\n✓ NDJSON lido automaticamente!")

### 4.3 Agregações em NDJSON

In [ ]:
# Análise por categoria
print("Análise por categoria:")
duckdb.sql("""
    SELECT 
        categoria,
        COUNT(*) AS total_produtos,
        SUM(estoque) AS estoque_total,
        AVG(preco) AS preco_medio,
        SUM(preco * estoque) AS valor_total_estoque
    FROM 'produtos.ndjson'
    GROUP BY categoria
    ORDER BY valor_total_estoque DESC
""").show()

## 5. Exportação para JSON

### 5.1 Exportação Básica

In [ ]:
# Criar conexão
con = duckdb.connect()

# Criar tabela
con.sql("""
    CREATE TABLE usuarios AS
    SELECT * FROM 'example.json'
""")

# Exportar usando COPY TO
con.sql("""
    COPY (
        SELECT * FROM usuarios WHERE salario > 6000
    ) TO 'salarios_altos.json' (FORMAT JSON, ARRAY true)
""")

print("✓ Arquivo 'salarios_altos.json' criado!")
print(f"Tamanho: {os.path.getsize('salarios_altos.json'):,} bytes")

# Verificar conteúdo
print("\nConteúdo:")
with open('salarios_altos.json', 'r', encoding='utf-8') as f:
    dados = json.load(f)
    print(json.dumps(dados[:2], ensure_ascii=False, indent=2))

### 5.2 Exportação como NDJSON

In [ ]:
# Exportar como NDJSON usando COPY
con.sql("""
    COPY (
        SELECT nome, idade, cidade
        FROM usuarios
        ORDER BY idade
    ) TO 'resumo.ndjson' (FORMAT JSON, ARRAY false)
""")

print("✓ Arquivo 'resumo.ndjson' criado!")

# Verificar formato NDJSON
print("\nConteúdo (cada linha é um JSON):")
with open('resumo.ndjson', 'r', encoding='utf-8') as f:
    for i, linha in enumerate(f, 1):
        if i <= 3:
            print(f"Linha {i}: {linha.strip()}")

## 6. Trabalhando com Estruturas Aninhadas

JSON pode ter objetos dentro de objetos.

### 6.1 Criar JSON Aninhado

In [ ]:
# JSON com estruturas aninhadas
dados_aninhados = [
    {
        "id": 1,
        "nome": "Alice",
        "endereco": {
            "rua": "Av. Paulista",
            "numero": 1000,
            "cidade": "São Paulo",
            "uf": "SP"
        },
        "contatos": {
            "email": "alice@email.com",
            "telefone": "11-98765-4321"
        }
    },
    {
        "id": 2,
        "nome": "Bob",
        "endereco": {
            "rua": "Av. Atlântica",
            "numero": 500,
            "cidade": "Rio de Janeiro",
            "uf": "RJ"
        },
        "contatos": {
            "email": "bob@email.com",
            "telefone": "21-99876-5432"
        }
    }
]

with open('pessoas_nested.json', 'w', encoding='utf-8') as f:
    json.dump(dados_aninhados, f, ensure_ascii=False, indent=2)

print("✓ Arquivo 'pessoas_nested.json' criado!")

# Mostrar estrutura
print("\nEstrutura aninhada:")
with open('pessoas_nested.json', 'r', encoding='utf-8') as f:
    print(f.read()[:400] + "...")

### 6.2 Acessar Campos Aninhados

In [ ]:
# Acessar campos aninhados com notação de ponto
print("Acessando campos aninhados:")
duckdb.sql("""
    SELECT 
        id,
        nome,
        endereco.cidade AS cidade,
        endereco.uf AS uf,
        contatos.email AS email,
        contatos.telefone AS telefone
    FROM 'pessoas_nested.json'
""").show()

print("\n✓ Campos aninhados extraídos!")

### 6.3 Filtrar por Campos Aninhados

In [ ]:
# Filtrar usando campos aninhados
print("Pessoas de São Paulo:")
duckdb.sql("""
    SELECT 
        nome,
        endereco.rua AS rua,
        endereco.cidade AS cidade
    FROM 'pessoas_nested.json'
    WHERE endereco.cidade = 'São Paulo'
""").show()

## 7. JSON de Dicionários Python

DuckDB pode consultar diretamente estruturas Python!

### 7.1 Consultar Lista de Dicionários

In [ ]:
# Criar dados em Python
dados_dict = [
    {"name": "Alice", "age": 30, "city": "São Paulo"},
    {"name": "Bob", "age": 25, "city": "Rio de Janeiro"},
    {"name": "Charlie", "age": 35, "city": "Belo Horizonte"},
    {"name": "Diana", "age": 28, "city": "Curitiba"}
]

# Salvar como JSON temporário e consultar
import json
with open('temp_python_data.json', 'w', encoding='utf-8') as f:
    json.dump(dados_dict, f)

# Consultar diretamente!
print("Consultando dados Python (via JSON):")
result = duckdb.sql("SELECT * FROM 'temp_python_data.json'")
result.show()

print("\n✓ DuckDB consulta estruturas Python via JSON!")

### 7.2 Agregações em Dados Python

In [ ]:
# Fazer agregações
print("Estatísticas:")
duckdb.sql("""
    SELECT 
        COUNT(*) AS total,
        AVG(age) AS idade_media,
        MIN(age) AS idade_min,
        MAX(age) AS idade_max
    FROM 'temp_python_data.json'
""").show()

# Por cidade
print("\nPor cidade:")
duckdb.sql("""
    SELECT city, COUNT(*) AS quantidade
    FROM 'temp_python_data.json'
    GROUP BY city
""").show()

## 8. Arrays e Listas em JSON

### 8.1 JSON com Arrays

In [ ]:
# JSON com arrays
dados_com_arrays = [
    {
        "id": 1,
        "nome": "Alice",
        "skills": ["Python", "SQL", "DuckDB"],
        "notas": [8.5, 9.0, 7.5]
    },
    {
        "id": 2,
        "nome": "Bob",
        "skills": ["JavaScript", "React", "Node.js"],
        "notas": [9.0, 8.0, 8.5]
    },
    {
        "id": 3,
        "nome": "Charlie",
        "skills": ["Java", "Spring"],
        "notas": [7.0, 8.0]
    }
]

with open('pessoas_arrays.json', 'w', encoding='utf-8') as f:
    json.dump(dados_com_arrays, f, ensure_ascii=False, indent=2)

print("✓ Arquivo 'pessoas_arrays.json' criado!")

# Ler dados
print("\nDados com arrays:")
duckdb.sql("SELECT * FROM 'pessoas_arrays.json'").show()

### 8.2 Trabalhar com Arrays

In [ ]:
# Funções de array
print("Análise de arrays:")
duckdb.sql("""
    SELECT 
        nome,
        len(skills) AS total_skills,
        skills[1] AS primeira_skill,
        list_avg(notas) AS media_notas,
        list_max(notas) AS melhor_nota
    FROM 'pessoas_arrays.json'
""").show()

### 8.3 UNNEST (Expandir Arrays)

In [ ]:
# Expandir arrays em linhas
print("Expandindo skills (cada skill vira uma linha):")
duckdb.sql("""
    SELECT 
        nome,
        UNNEST(skills) AS skill
    FROM 'pessoas_arrays.json'
    ORDER BY nome, skill
""").show()

print("\n✓ Arrays expandidos com UNNEST!")

## 9. Consultas Complexas em JSON

### 9.1 JOIN entre Arquivos JSON

In [ ]:
# Criar segundo arquivo JSON
vendas = [
    {"id_usuario": 1, "produto": "Notebook", "valor": 3500.00},
    {"id_usuario": 1, "produto": "Mouse", "valor": 50.00},
    {"id_usuario": 2, "produto": "Teclado", "valor": 150.00},
    {"id_usuario": 3, "produto": "Monitor", "valor": 800.00}
]

with open('vendas.json', 'w') as f:
    json.dump(vendas, f, indent=2)

# JOIN entre arquivos
print("JOIN entre usuários e vendas:")
duckdb.sql("""
    SELECT 
        u.nome,
        u.cidade,
        v.produto,
        v.valor
    FROM 'example.json' u
    JOIN 'vendas.json' v ON u.id = v.id_usuario
    ORDER BY u.nome, v.valor DESC
""").show()

### 9.2 Agregações Complexas

In [ ]:
# Total de vendas por usuário
print("Total de vendas por usuário:")
duckdb.sql("""
    SELECT 
        u.nome,
        u.cidade,
        COUNT(v.produto) AS total_compras,
        SUM(v.valor) AS valor_total
    FROM 'example.json' u
    LEFT JOIN 'vendas.json' v ON u.id = v.id_usuario
    GROUP BY u.nome, u.cidade
    ORDER BY valor_total DESC NULLS LAST
""").show()

## 10. Conversão JSON ↔ CSV ↔ Parquet

### 10.1 JSON → CSV

In [ ]:
# Converter JSON para CSV
duckdb.sql("""
    COPY (
        SELECT * FROM 'example.json'
    ) TO 'usuarios.csv' (HEADER)
""")

print("✓ Convertido JSON → CSV")
print(f"Tamanho CSV: {os.path.getsize('usuarios.csv'):,} bytes")

# Verificar
print("\nPrimeiras linhas do CSV:")
duckdb.sql("SELECT * FROM 'usuarios.csv' LIMIT 3").show()

### 10.2 JSON → Parquet

In [ ]:
# Converter JSON para Parquet
duckdb.sql("""
    COPY (
        SELECT * FROM 'example.json'
    ) TO 'usuarios.parquet'
""")

print("✓ Convertido JSON → Parquet")
print(f"Tamanho Parquet: {os.path.getsize('usuarios.parquet'):,} bytes")

# Verificar
print("\nPrimeiras linhas do Parquet:")
duckdb.sql("SELECT * FROM 'usuarios.parquet' LIMIT 3").show()

### 10.3 Comparação de Tamanhos

In [ ]:
# Comparar tamanhos dos formatos
json_size = os.path.getsize('example.json')
csv_size = os.path.getsize('usuarios.csv')
parquet_size = os.path.getsize('usuarios.parquet')

print("Comparação de tamanhos (mesmos dados):")
print("=" * 40)
print(f"JSON:    {json_size:>6,} bytes (100%)")
print(f"CSV:     {csv_size:>6,} bytes ({csv_size/json_size*100:.1f}%)")
print(f"Parquet: {parquet_size:>6,} bytes ({parquet_size/json_size*100:.1f}%)")
print("\n💡 Parquet é o formato mais compacto!")

## 11. Performance e Boas Práticas

### 📊 Dicas de Performance:

1. **Use NDJSON para grandes volumes**: Mais eficiente que JSON array
2. **Evite estruturas muito aninhadas**: Dificulta queries
3. **Considere Parquet**: Melhor performance para análise
4. **Use JSON para APIs**: Formato universal
5. **Filtre cedo**: WHERE antes de processar
6. **Converta tipos**: JSON preserva tipos, mas valide

### 11.1 Benchmark: JSON vs NDJSON

In [ ]:
import time

# Criar datasets grandes
print("Criando datasets de teste...")

# JSON Array
con.sql("""
    COPY (
        SELECT 
            i AS id,
            'Usuario_' || i AS nome,
            20 + (i % 50) AS idade
        FROM range(1, 10001) t(i)
    ) TO 'large.json' (FORMAT JSON, ARRAY true)
""")

# NDJSON
con.sql("""
    COPY (
        SELECT 
            i AS id,
            'Usuario_' || i AS nome,
            20 + (i % 50) AS idade
        FROM range(1, 10001) t(i)
    ) TO 'large.ndjson' (FORMAT JSON, ARRAY false)
""")

json_size = os.path.getsize('large.json')
ndjson_size = os.path.getsize('large.ndjson')

print(f"\nJSON array:  {json_size:>10,} bytes")
print(f"NDJSON:      {ndjson_size:>10,} bytes")

# Benchmark leitura
print("\nBenchmark de leitura (10k linhas):")
print("=" * 40)

# JSON
start = time.time()
result1 = duckdb.sql("SELECT COUNT(*) FROM 'large.json'").fetchall()
time_json = time.time() - start

# NDJSON
start = time.time()
result2 = duckdb.sql("SELECT COUNT(*) FROM 'large.ndjson'").fetchall()
time_ndjson = time.time() - start

print(f"JSON:    {time_json*1000:>8.2f}ms")
print(f"NDJSON:  {time_ndjson*1000:>8.2f}ms")

if time_json < time_ndjson:
    print(f"\nJSON é {time_ndjson/time_json:.1f}x mais rápido!")
else:
    print(f"\nNDJSON é {time_json/time_ndjson:.1f}x mais rápido!")

### 11.2 Quando Usar Cada Formato

| Formato | Quando Usar |
|---------|-------------|
| **JSON Array** | APIs, configurações, dados pequenos |
| **NDJSON** | Logs, streams, dados grandes |
| **CSV** | Compatibilidade, dados tabulares simples |
| **Parquet** | Análise, data lakes, grandes volumes |

### Recomendação:
- 🌐 **JSON**: APIs, web services, configurações
- 📝 **NDJSON**: Logs, streaming, processamento incremental
- 📊 **Parquet**: Análise de dados, data warehouses

## 🎯 Resumo do Capítulo

Neste capítulo, exploramos:

1. ✅ **JSON Array**: Formato padrão com array de objetos
2. ✅ **NDJSON**: Cada linha um objeto (melhor para grandes volumes)
3. ✅ **Leitura**: `read_json()` e SQL direto
4. ✅ **Exportação**: `write_json()` e `COPY TO`
5. ✅ **Estruturas aninhadas**: Objetos dentro de objetos
6. ✅ **Dicionários Python**: DuckDB consulta diretamente!
7. ✅ **Arrays**: Trabalhar com listas em JSON
8. ✅ **UNNEST**: Expandir arrays em linhas
9. ✅ **JOINs**: Combinar múltiplos arquivos JSON
10. ✅ **Conversões**: JSON ↔ CSV ↔ Parquet
11. ✅ **Performance**: Comparação entre formatos

### 🔑 Pontos-Chave:
- DuckDB lê JSON **nativamente**
- Suporta **estruturas aninhadas**
- **NDJSON** para grandes volumes
- Consulta **dicionários Python** diretamente
- **UNNEST** para expandir arrays

### 📚 Próximo Capítulo:
Integração com Python!

## 🧹 Limpeza (Opcional)

In [ ]:
# Arquivos para remover
arquivos = [
    'example.json', 'produtos.ndjson', 'salarios_altos.json', 'resumo.ndjson',
    'pessoas_nested.json', 'pessoas_arrays.json', 'vendas.json',
    'usuarios.csv', 'usuarios.parquet', 'large.json', 'large.ndjson'
]

for arquivo in arquivos:
    if os.path.exists(arquivo):
        os.remove(arquivo)
        print(f"✓ Removido: {arquivo}")

print("\n✓ Limpeza concluída!")